In [2]:
import os
import json

import gymnasium as gym
import logic_gym
from logic_gym.wrappers.text_to_box_wrapper import TextToBoxWrapper

from stable_baselines3 import PPO
from sb3_contrib import MaskablePPO

MAX_PROOF_LENGTH = 16
base_env = gym.make("logic_gym/LogicGym-v0", max_proof_length=MAX_PROOF_LENGTH)
env = TextToBoxWrapper(base_env)

# Load the long task

file_path = os.path.join("dataprep", "1hop_trueontology_formalized.json")
with open(file_path, "r") as file:
    tasks = open(file_path)
json_tasks = json.load(tasks)
task = json_tasks['example1']['test_example']['formalized']
    
env.unwrapped.set_task(task)
# print(env.unwrapped.get_task())

agent = MaskablePPO("MlpPolicy", env, verbose=1, gamma=0.4, seed=32)
agent.learn(total_timesteps=70000)
agent.save("ppo_logic_gym")



Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/akash/anaconda3/envs/fol/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_masks to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_masks` for environment variables or `env.get_wrapper_attr('action_masks')` that will search the reminding wrappers.
  logger.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.1     |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 90       |
|    iterations      | 1        |
|    time_elapsed    | 22       |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 22.4       |
|    ep_rew_mean          | 0.01       |
| time/                   |            |
|    fps                  | 85         |
|    iterations           | 2          |
|    time_elapsed         | 47         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01268466 |
|    clip_fraction        | 0.125      |
|    clip_range           | 0.2        |
|    entropy_loss         | -6.13      |
|    explained_variance   | -56.287483 |
|    learning_rate        | 0.0003     |
|   

In [5]:
import gymnasium as gym
import logic_gym
from logic_gym.wrappers.text_to_box_wrapper import TextToBoxWrapper


from sb3_contrib import MaskablePPO
from stable_baselines3 import PPO

base_env = gym.make("logic_gym/LogicGym-v0", max_proof_length=MAX_PROOF_LENGTH)
# Load the long task
env = TextToBoxWrapper(base_env)
env.unwrapped.set_task(task)

agent = MaskablePPO.load("ppo_logic_gym", env=env)

NUM_ATTEMPTS = 500

for i in range(NUM_ATTEMPTS):
    observation, info = env.reset()
    terminated, truncated = False, False
    while not terminated and not truncated:
        action, _ = agent.predict(observation)  # agent policy that uses the observation and info
        observation, reward, terminated, truncated, info = env.step(action)
    if(i%25 == 0):
        print(env.unwrapped.get_stats())
    if terminated:
        print("found a proof on attempt", i)
        break

print(terminated, truncated)
# print(observation)
print(env.unwrapped.get_state_for_humans())
print(env.unwrapped.get_stats())
env.close()


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
{'reset_count': 1, 'terminated_count': 0, 'truncated_count': 1, 'steps': 32}
found a proof on attempt 5
True False
Ax.(Cat(x) -> Feline(x))                                      (0)  Given
Ax.(Feline(x) -> Carnivore(x))                                (1)  Given
Ax.(Carnivore(x) -> ~Herbivorous(x))                          (2)  Given
Ax.(Snake(x) -> ~WarmBlooded(x))                              (3)  Given
Ax.(Carnivore(x) -> Mammal(x))                                (4)  Given
Ax.(Mammal(x) -> WarmBlooded(x))                              (5)  Given
Ax.(Mammal(x) -> Vertebrate(x))                               (6)  Given
Ax.(Vertebrate(x) -> Animal(x))                               (7)  Given
Ax.(Animal(x) -> Multicellular(x))                            (8)  Given
Mammal(Sally)                                                 (9)  Given
WarmBlooded(Sally)                                           (10)  Goal
Mammal